This scripts applies the k-fold CV.  
First it gets the list of all videos that have at least one frame that has 3 subscores, then divide videos int n fold, then apply CV.

In [28]:
import os
from datetime import datetime
import time
import numpy as np
import random
from statistics import mean
from tqdm import tqdm
from random import shuffle
import torch
import torchvision
import torchvision.transforms as transforms
from torch.optim import lr_scheduler
from datasets.UCEIS_dataset import UCEIS_cord
from models import resnet, densenet
from utils.losses import multitask_CE
from utils.provider import true_predictions, true_predictions_UCEIS, fix_seed, get_dataset_mean_and_std, convert_multiclass_to_binary_class
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, cohen_kappa_score
from utils.detect_blob import DetectBlob
import wandb

In [29]:
fix_seed(35)

learning_rate = 0.0001
weight_decay = 0.001
num_epoch = 100 
best_acc = 0
best_threshold = 0.0001
batch_size = 24
num_worker = 4
early_stop_counter = 0
scheduler_patience = 5
early_stopping_thresh = 10
enable_wandb = True
optimizer_name = "Adam"
model_name = "ResNet34"

In [30]:
frame_root =  "../IBD/data_all_frames_540x960"
cord_project_ID = '499f6b35-de54-4e23-82ac-19420fabd3c0'
cord_API_key = 'mt-zFQiptFF4ytFlfenYgbEbCTEXpkzJjMBcwGCBwyM'

folders = [folder for folder in os.listdir(frame_root) if not folder.startswith(".")]
dataset = UCEIS_cord(folders, frame_root, cord_project_ID, cord_API_key)

print("total labeled videos: ", len(dataset.labeled_video_names))
print(sorted(dataset.labeled_video_names))

[06/27/2021 08:29:18 PM] [INFO] [configs.py:65] [__init__()] Initialising Cord client with endpoint: https://api.cord.tech/public and resource_id: 499f6b35-de54-4e23-82ac-19420fabd3c0
[06/27/2021 08:29:18 PM] [INFO] [querier.py:97] [execute()] Request: {"query_type": "apikeymeta", "query_method": "GET", "values": {"uid": null, "payload": null}}
[06/27/2021 08:29:18 PM] [INFO] [client.py:111] [initialise_with_config()] Initialising Cord client for project using key: New read key
[06/27/2021 08:29:18 PM] [INFO] [querier.py:97] [execute()] Request: {"query_type": "project", "query_method": "GET", "values": {"uid": null, "payload": null}}
[06/27/2021 08:29:18 PM] [INFO] [querier.py:97] [execute()] Request: {"query_type": "labelrow", "query_method": "GET", "values": {"uid": "26b3ce1b-bfa2-429b-9c0e-6270f78..
[06/27/2021 08:29:19 PM] [INFO] [querier.py:97] [execute()] Request: {"query_type": "labelrow", "query_method": "GET", "values": {"uid": "7e73964e-9a63-4024-8f40-19485ca..
[06/27/2021 0

In [31]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device: ", device)

device:  cuda:0


In [32]:
def write_results_to_file(subscore_vascular, subscore_bleeding, subscore_erosion, score_UCEIS, QWKs, acc_remissions, kappa_scores):
    results = []
    
    results.append("subscore_vascular: " + str(subscore_vascular))
    results.append("subscore_vascular mean: " + str(mean(subscore_vascular)))
    results.append("")
    
    results.append("subscore_bleeding: " + str(subscore_bleeding))
    results.append("subscore_bleeding mean: " + str(mean(subscore_bleeding)))
    results.append("")
    
    results.append("subscore_erosion: " + str(subscore_erosion))
    results.append("subscore_erosion mean: " + str(mean(subscore_erosion)))
    results.append("")

    results.append("score_UCEIS: " + str(score_UCEIS))
    results.append("score_UCEIS mean: " + str(mean(score_UCEIS)))
    results.append("")

    results.append("QWKs: " + str(QWKs))
    results.append("QWKs mean: " + str(mean(QWKs)))
    results.append("")

    results.append("acc_remissions: " + str(acc_remissions))
    results.append("acc_remissions mean: " + str(mean(acc_remissions)))
    results.append("")

    results.append("kappa_scores: " + str(kappa_scores))
    results.append("kappa_scores mean: " + str(mean(kappa_scores)))
    results.append("")
    
    results_separated = map(lambda x: x + "\n", results)
    file = open(os.path.join(wandb.run.dir, "results.txt"), "w")
    file.writelines(results_separated)
    file.close()

In [33]:
def train(model, device, train_loader, criterion, optimizer):
    model.train()
    training_loss = 0.0
    correct_subscores = 0
    correct_UCEIS = 0

    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        target.transpose_(0, 1)
        
        output = model(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        correct_subscores += true_predictions(output, target)
        correct_UCEIS += true_predictions_UCEIS(output, target)

        training_loss += loss.item()

    training_loss /= len(train_loader)
    correct_subscores /= (len(train_loader.dataset) * 3)
    correct_UCEIS /= len(train_loader.dataset)

    return (training_loss, correct_subscores, correct_UCEIS)

def validation(model, device, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct_subscores = 0
    correct_UCEIS = 0

    with torch.no_grad():
        for data, target, in val_loader:
            data, target = data.to(device), target.to(device)
            target.transpose_(0, 1)

            output = model(data)
            loss = criterion(output, target)
            correct_subscores += true_predictions(output, target)
            correct_UCEIS += true_predictions_UCEIS(output, target)

            val_loss += loss.item()

        val_loss /= len(val_loader)
        correct_subscores /= (len(val_loader.dataset) * 3)
        correct_UCEIS /= len(val_loader.dataset)

        return (val_loss, correct_subscores, correct_UCEIS)

In [34]:
def run_experiment(experiment_id, train_folders, val_folders, normalize, best_acc, early_stop_counter):
        
    train_transform = transforms.Compose([transforms.RandomResizedCrop((540, 960), scale=(0.5, 1.5), ratio=(0.75, 1.33)),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.RandomRotation((-180, 180)),
                                          transforms.ToTensor(),
                                          normalize])
       
    val_transform = transforms.Compose([transforms.ToTensor(),
                                        normalize])
    
    train_dataset = UCEIS_cord(train_folders, frame_root, cord_project_ID, cord_API_key, transform=train_transform)
    val_dataset = UCEIS_cord(val_folders, frame_root, cord_project_ID, cord_API_key, transform=val_transform)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                                               num_workers=num_worker,
                                               pin_memory=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_worker,
                                             pin_memory=True)
    
    print("Images in train set: ", len(train_dataset))
    print("Images in val set: ", len(val_dataset))
    
    if model_name == "ResNet18":
        model = resnet.resnet18(pretrained=True, num_classes=[3, 4, 4])
    elif model_name == "ResNet34":
        model = resnet.resnet34(pretrained=True, num_classes=[3, 4, 4])
    elif model_name == "ResNet50":
        model = resnet.resnet50(pretrained=True, num_classes=[3, 4, 4])
    elif model_name == "DenseNet201":
        model = densenet.densenet201(pretrained=True, num_classes=[3, 4, 4])
    elif model_name == "DenseNet121":
        model = densenet.densenet121(pretrained=True, num_classes=[3, 4, 4])

    model.to(device)

    criterion = multitask_CE()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=0.2, patience=scheduler_patience, threshold=best_threshold,
                                               verbose=True)

    experiment_signature = str(model_name) + " ID=" + str(experiment_id) + " lr=" + str(learning_rate) + " reg=" + str(weight_decay) + " bs=" + str(batch_size)    
    print("model: " + experiment_signature + " worker: " + str(num_worker))
    
    last_epoch = 0
    for epoch in tqdm(range(num_epoch)):        
        last_epoch = epoch

        train_loss, train_accuracy_subscore, train_accuracy_UCEIS = train(model, device, train_loader, criterion, optimizer)
        val_loss, val_accuracy_subscore, val_accuracy_UCEIS = validation(model, device, val_loader, criterion)
        scheduler.step(val_accuracy_UCEIS)
        
        if enable_wandb:
            wandb.log(
                    {"epoch"     : epoch + 1,
                     "lr"        : optimizer.param_groups[0]['lr'],
                     'train loss': train_loss,
                     'val loss'  : val_loss,
                     'train acc subscore' : train_accuracy_subscore,
                     'val acc subscore'   : val_accuracy_subscore,
                     'train acc UCEIS' : train_accuracy_UCEIS,
                     'val acc UCEIS'   : val_accuracy_UCEIS})
        
        if (val_accuracy_UCEIS > best_acc * (1 + best_threshold)):
            early_stop_counter = 0
            best_acc = val_accuracy_UCEIS
            if enable_wandb:
                wandb.run.summary["best accuracy UCEIS"] = best_acc            
            torch.save(model.state_dict(), "weights/4_best_" + model_name + '_' + str(experiment_id) +'.pth.tar')
        else:
            early_stop_counter += 1

        if early_stop_counter >= early_stopping_thresh:
            print("Early stopping at: " + str(epoch))
            break

    print("Fold: " + str(experiment_id) + ", best validation set accuracy: " + str(best_acc))

In [35]:
def get_test_results(experiment_id, test_folders, normalize):
    test_transform = transforms.Compose([transforms.ToTensor(),
                                         normalize])

    test_dataset = UCEIS_cord(test_folders, frame_root, cord_project_ID, cord_API_key, transform=test_transform)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=num_worker,
                                              pin_memory=True)
    
    print("Images in val set: ", len(test_dataset))
    
    state_dict_name = "weights/4_best_" + model_name + '_' + str(experiment_id) +'.pth.tar'
    if model_name == "ResNet18":
        model = resnet.resnet18(num_classes=[3, 4, 4])
    elif model_name == "ResNet34":
        model = resnet.resnet34(num_classes=[3, 4, 4])
    elif model_name == "ResNet50":
        model = resnet.resnet50(num_classes=[3, 4, 4])
    elif model_name == "DenseNet201":
        model = densenet.densenet201(num_classes=[3, 4, 4])
    elif model_name == "DenseNet121":
        model = densenet.densenet121(num_classes=[3, 4, 4])

    model.load_state_dict(torch.load(state_dict_name, map_location=device))
    model.to(device)
    model.eval()
    
    y_true_UCEIS = []
    y_true_vascular = []
    y_true_bleeding = []
    y_true_erosion = []

    y_pred_UCEIS = []
    y_pred_vascular = []
    y_pred_bleeding = []
    y_pred_erosion = []

    with torch.no_grad():
        for id, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            target.transpose_(0, 1)

            y_true_vascular.append(target[0].item())
            y_true_bleeding.append(target[1].item())
            y_true_erosion.append(target[2].item())
            y_true_UCEIS.append(target.sum().item())

            output = model(data)

            prediction_0 = output[0].argmax(dim=1, keepdim=True)[0][0].item()
            y_pred_vascular.append(prediction_0)

            prediction_1 = output[1].argmax(dim=1, keepdim=True)[0][0].item()
            y_pred_bleeding.append(prediction_1)

            prediction_2 = output[2].argmax(dim=1, keepdim=True)[0][0].item()
            y_pred_erosion.append(prediction_2)

            y_pred_UCEIS.append(prediction_0 + prediction_1 + prediction_2)
    
    # Scores
    accuracy_vascular = accuracy_score(y_true_vascular, y_pred_vascular)
    accuracy_bleeding = accuracy_score(y_true_bleeding, y_pred_bleeding)
    accuracy_erosion = accuracy_score(y_true_erosion, y_pred_erosion)
    accuracy_UCEIS = accuracy_score(y_true_UCEIS, y_pred_UCEIS)
    
    quadratic_kappa_score = cohen_kappa_score(y_true_UCEIS, y_pred_UCEIS, weights="quadratic")
    
    y_true_remission = convert_multiclass_to_binary_class(y_true_UCEIS)
    y_pred_remission = convert_multiclass_to_binary_class(y_pred_UCEIS)
    accuracy_UCEIS_remission = accuracy_score(y_true_remission, y_pred_remission)
    
    kappa_score = cohen_kappa_score(y_true_remission, y_pred_remission)
    
    
    return accuracy_vascular, accuracy_bleeding, accuracy_erosion, accuracy_UCEIS, quadratic_kappa_score, accuracy_UCEIS_remission, kappa_score

In [36]:
number_of_experiments = 5

train_folders_list = []
val_folders_list = []

labeled_videos = dataset.labeled_video_names 

shuffle(labeled_videos)
video_folds = np.array_split(labeled_videos, number_of_experiments)

for i in range(number_of_experiments):
    # Get train videos
    selected_folds = []
    for k in range(number_of_experiments):
        if k != i:
            selected_folds.extend(video_folds[k])
    train_folders_list.append(selected_folds)
    
    # Get val videos
    val_folders_list.append(video_folds[i])
    
    print("\nFold ", i+1)
    print("Train folders: "+str(train_folders_list[i]))
    print("Val folders: "+str(val_folders_list[i]))        


Fold  1
Train folders: ['UC7-4.mp4', 'UC52-3.mp4', 'UC8-6.mp4', 'UC58-4.mp4', 'UC36-0.mp4', 'UC51-7.mp4', 'UC81-4.mp4', 'UC62-5.mp4', 'UC73-5.mp4', 'UC39-4.mp4', 'UC2-1.mp4', 'UC9-4.mp4', 'UC63-2.mp4', 'UC22-0.mp4', 'UC15-1.mp4', 'UC18-4.mp4', 'UC87-6.mp4', 'UC76-4.mp4', 'UC4-2.mp4', 'UC17-4.mp4', 'UC42-7.mp4', 'UC72-1.mp4', 'UC49-4.mp4', 'UC70-2.mp4', 'UC80-2.mp4', 'UC5-4.mp4', 'UC77-3.mp4', 'UC23-1.mp4', 'UC21-2.mp4', 'UC66-0.mp4', 'UC41-4.mp4', 'UC48-6.mp4', 'UC90-3.mp4', 'UC3-5.mp4', 'UC64-1.mp4', 'UC35-5.mp4', 'UC65-4.mp4', 'UC44-5.mp4', 'UC47-4.mp4', 'UC85-0.mp4', 'UC24-4.mp4', 'UC30-5.mp4', 'UC57-0.mp4', 'UC13-0.mp4', 'UC88-1.mp4', 'UC38-5.mp4', 'UC46-0.mp4', 'UC68-1.mp4', 'UC20-1.mp4', 'UC10-0.mp4', 'UC53-4.mp4', 'UC14-4.mp4', 'UC60-1.mp4', 'UC25-4.mp4', 'UC74-2.mp4', 'UC79-4.mp4', 'UC31-0.mp4', 'UC54-4.mp4']
Val folders: ['UC69-7.mp4' 'UC61-4.mp4' 'UC71-1.mp4' 'UC29-5.mp4' 'UC16-4.mp4'
 'UC67-5.mp4' 'UC19-1.mp4' 'UC55-5.mp4' 'UC28-0.mp4' 'UC1-6.mp4'
 'UC59-5.mp4' 'UC32-0.mp4'

In [ ]:
subscore_vascular = []
subscore_bleeding = []
subscore_erosion = []
score_UCEIS = []
QWKs = []
acc_remissions = []
kappa_scores = []


if enable_wandb:
    group_id = wandb.util.generate_id()
        
for i in range(number_of_experiments):
    id = i+1    
    
    if enable_wandb:
        wandb.init(project="cord_uceis",
                   group="CV_2_" + group_id,
                   save_code=True,
                   reinit=True)
        wandb.run.name = "epoch_" + str(id)
        wandb.run.save()

        config = wandb.config
        config.model = model_name
        config.dataset = "37k"
        config.lr = learning_rate
        config.wd = weight_decay
        config.bs = batch_size
        config.num_worker = num_worker
        config.project = "classification"
        config.optimizer = optimizer_name
    
    # ----- TRAIN -----    
    best_acc = 0
    early_stop_counter = 0
        
    print("\n--------------> Starting Fold " + str(id))
    start = time.time()
    
    train_dataset = UCEIS_cord(train_folders_list[i], frame_root, cord_project_ID, cord_API_key)
    train_set_mean, train_set_std = get_dataset_mean_and_std(train_dataset)
    normalize = transforms.Normalize(mean=train_set_mean,
                                     std=train_set_std)
    
    run_experiment(id, train_folders_list[i], val_folders_list[i], normalize, best_acc, early_stop_counter)
    elapsed = time.time() - start
    print("total training time: " + str(elapsed / 60) + " min")    
    
    # ----- TEST -----
    acc_vascular, acc_bleeding, acc_erosion, acc_UCEIS, QWK, acc_remission, kappa_score = get_test_results(id, val_folders_list[i], normalize)
    subscore_vascular.append(acc_vascular)
    subscore_bleeding.append(acc_bleeding)
    subscore_erosion.append(acc_erosion)
    score_UCEIS.append(acc_UCEIS)
    QWKs.append(QWK)
    acc_remissions.append(acc_remission)
    kappa_scores.append(kappa_score)
    
    print("---------------> Fold " + str(id) + " finished!")
    
    if enable_wandb:
        if id == number_of_experiments:
            # Write CV results to the last fold's files
            wandb.run.summary["QWK"] = mean(QWKs)
            wandb.run.summary["Acc Remission"] = mean(acc_remissions)
            write_results_to_file(subscore_vascular, subscore_bleeding, subscore_erosion, score_UCEIS, QWKs, acc_remissions, kappa_scores)
        wandb.run.finish()


wandb: You can sync this run to the cloud by running:
wandb: wandb sync /home/jupyter/gi/Colab Notebooks/wandb/offline-run-20210627_195539-1rkqx0qb


wandb: W&B syncing is set to `offline` in this directory.  Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.
wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.



--------------> Starting Fold 1
[06/27/2021 08:29:53 PM] [INFO] [configs.py:65] [__init__()] Initialising Cord client with endpoint: https://api.cord.tech/public and resource_id: 499f6b35-de54-4e23-82ac-19420fabd3c0
[06/27/2021 08:29:53 PM] [INFO] [querier.py:97] [execute()] Request: {"query_type": "apikeymeta", "query_method": "GET", "values": {"uid": null, "payload": null}}
[06/27/2021 08:29:53 PM] [INFO] [client.py:111] [initialise_with_config()] Initialising Cord client for project using key: New read key
[06/27/2021 08:29:53 PM] [INFO] [querier.py:97] [execute()] Request: {"query_type": "project", "query_method": "GET", "values": {"uid": null, "payload": null}}
[06/27/2021 08:29:53 PM] [INFO] [querier.py:97] [execute()] Request: {"query_type": "labelrow", "query_method": "GET", "values": {"uid": "dfef2edd-4467-4705-a38b-7c2e977..
[06/27/2021 08:29:53 PM] [INFO] [querier.py:97] [execute()] Request: {"query_type": "labelrow", "query_method": "GET", "values": {"uid": "e21d53b3-e89a-

  9%|▉         | 9/100 [7:34:14<76:24:19, 3022.63s/it]

Epoch     9: reducing learning rate of group 0 to 2.0000e-05.


...

epoch,▁▂▂▃▃▄▅▅▆▆▇▇█
lr,████████▁▁▁▁▁
train loss,█▅▄▃▃▃▃▃▃▁▁▁▁
val loss,█▃▆▁▇▅▂▆▁▄▃▆▆
train acc subscore,▁▅▅▆▆▆▆▆▇████
val acc subscore,▃▄█▅▆▁▄▁▆▃▆▂▂
train acc UCEIS,▁▄▅▅▆▆▆▆▆████
val acc UCEIS,▄▅█▇▆▂▆▂▁▇▄▅▄
_runtime,▁▂▂▃▃▄▅▅▆▆▇▇█
_timestamp,▁▂▂▃▃▄▅▅▆▆▇▇█
_step,▁▂▂▃▃▄▅▅▆▆▇▇█


wandb: You can sync this run to the cloud by running:
wandb: wandb sync /home/jupyter/gi/Colab Notebooks/wandb/offline-run-20210627_202947-gw7x35u3
wandb: W&B syncing is set to `offline` in this directory.  Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.
wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.



--------------> Starting Fold 2
[06/28/2021 07:42:22 AM] [INFO] [configs.py:65] [__init__()] Initialising Cord client with endpoint: https://api.cord.tech/public and resource_id: 499f6b35-de54-4e23-82ac-19420fabd3c0
[06/28/2021 07:42:22 AM] [INFO] [querier.py:97] [execute()] Request: {"query_type": "apikeymeta", "query_method": "GET", "values": {"uid": null, "payload": null}}
[06/28/2021 07:42:22 AM] [INFO] [client.py:111] [initialise_with_config()] Initialising Cord client for project using key: New read key
[06/28/2021 07:42:22 AM] [INFO] [querier.py:97] [execute()] Request: {"query_type": "project", "query_method": "GET", "values": {"uid": null, "payload": null}}
[06/28/2021 07:42:22 AM] [INFO] [querier.py:97] [execute()] Request: {"query_type": "labelrow", "query_method": "GET", "values": {"uid": "75958069-887a-4ca3-b763-78e4d9b..
[06/28/2021 07:42:22 AM] [INFO] [querier.py:97] [execute()] Request: {"query_type": "labelrow", "query_method": "GET", "values": {"uid": "efe6ab38-76da-

  0%|          | 0/100 [00:00<?, ?it/s]

model: ResNet34 ID=2 lr=0.0001 reg=0.001 bs=24 worker: 4


In [ ]:
print("Vascular pattern: "+str(subscore_vascular))
print("Vascular pattern (mean): "+ str(mean(subscore_vascular)))
print("")

print("Bleeding: "+str(subscore_bleeding))
print("Bleeding (mean): "+ str(mean(subscore_bleeding)))
print("")

print("Erosion: "+str(subscore_erosion))
print("Erosion (mean): "+ str(mean(subscore_erosion)))
print("")

print("UCEIS: "+str(score_UCEIS))
print("UCEIS (mean): "+ str(mean(score_UCEIS)))
print("")

print("QWK: "+str(QWKs))
print("QWKs (mean): "+ str(mean(QWKs)))
print("")

print("acc_remission: "+str(acc_remissions))
print("acc_remission (mean): "+ str(mean(acc_remissions)))
print("")

print("kappa_score: "+str(kappa_scores))
print("kappa_score (mean): "+ str(mean(kappa_scores)))
print("")


In [40]:
import gc
import torch

gc.collect()

torch.cuda.empty_cache()
gc.collect()


0

In [39]:
+2

4